In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from setup import *

%load_ext autoreload
%autoreload 2

In [2]:
hh = pd.read_csv(survey_dir+"household.csv")
pr = pd.read_csv(survey_dir+"person.csv")
pl = pd.read_csv(survey_dir+"place.csv")
pt = pd.read_csv(survey_dir+"place_transit.csv")
lc = pd.read_csv(survey_dir+"location.csv")

/home/jtl/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3072: DtypeWarning: Columns (45,50) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/home/jtl/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3072: DtypeWarning: Columns (11,20) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
num_pl = len(pl)
print(len(hh), "Households", len(pr), "People", len(pl), "Places", len(pl[pl.placeno!=1]), "Trips")

12391 Households 30683 People 128229 Places 99652 Trips


In [4]:
# Income
f_inc = (hh.hhinc >= 0) | (hh.hhinc2 >= -1)
print(np.sum(f_inc==0), 'households with no income')

30 households with no income


In [5]:
# Trip numbers
f_hhtrips = (hh.hhtrips>0)
print(np.sum(f_hhtrips==0), 'households with no trips')

349 households with no trips


In [6]:
hh = hh[f_inc & f_hhtrips]

In [7]:
# Age
f_age = (pr.aage >= -1)
print(np.sum(f_age==0), 'people with no age')

10 people with no age


In [8]:
# License
f_lic = (pr.lic >= -1)
print(np.sum(f_lic==0), 'people with no license info')

2 people with no license info


In [9]:
# Education
f_edu = (pr.educ > 0)
print(np.sum(f_edu==0), 'people with no education info')

42 people with no education info


In [10]:
pr = pr[f_age & f_lic & f_edu]

In [11]:
# Travel Mode
f_mode = (pl['mode'] >= -1) & (pl['mode'] // 100 < 8)
print(np.sum(f_mode==0), 'places with invalid modes (not available or air travel)')

331 places with invalid modes (not available or air travel)


In [12]:
pl = pl[f_mode]

In [13]:
# Trip Location
f_state_loc = (lc['state_fips'].isin([17,18,26,55]))
# tight restrictions to speed up the image download process
#f_state_loc = (lc['state_fips'].isin([17]))
pl = pd.merge(lc[f_state_loc][['sampno','locno']], pl, on=['sampno','locno'])

print(np.sum(f_state_loc==0), 'locations not in valid states')
print(num_pl-len(pl), 'places not in valid states')

1968 locations not in valid states
989 places not in valid states


In [14]:
lc = lc[f_state_loc]

In [15]:
print("Left with", len(hh), "Households", len(pr), "People", len(pl), "Places", len(pl[pl.placeno!=1]), "Trips")

Left with 12013 Households 30640 People 127240 Places 98899 Trips


## Turn Places into Trips

In [16]:
pl_new = pl[['sampno', 'perno', 'placeGroup', 'locno', 'arrtime', 'deptime', 'travtime', 'mode']].sort_values(by=['sampno','perno','placeGroup']).reset_index(drop=True)
pl_new['prev_placeGroup'] = pl_new['placeGroup'] - 1

In [17]:
tp = pd.merge(pl_new, pl_new, left_on=['sampno', 'perno', 'placeGroup'], right_on=['sampno', 'perno', 'prev_placeGroup'], suffixes=('_1','_2'))
tp = tp[['sampno', 'perno', 'placeGroup_1', 'placeGroup_2', 'locno_1', 'locno_2', 'deptime_1', 'arrtime_2', 'travtime_2', 'mode_2']]

In [18]:
lc = lc[['sampno', 'locno', 'loctype', 'state', 'country', 'state_fips',
       'county_fips', 'tract_fips', 'out_region', 'home', 'latitude',
       'longitude']]
tp = pd.merge(tp, lc, left_on=['sampno', 'locno_1'], right_on = ['sampno', 'locno'])
tp = pd.merge(tp, lc, left_on=['sampno', 'locno_2'], right_on = ['sampno', 'locno'], suffixes=("_1", "_2"))

In [19]:
tp = pd.merge(tp, pr[['sampno','perno','wtperfin']], on=['sampno','perno'])

In [20]:
tp.to_csv(data_dir+"trips.csv", index=False)

In [21]:
tp['tract_1'] = tp['state_fips_1']*1000000000+tp['county_fips_1']*1000000+tp['tract_fips_1']
tp['tract_2'] = tp['state_fips_2']*1000000000+tp['county_fips_2']*1000000+tp['tract_fips_2']

In [22]:
tp.columns

Index(['sampno', 'perno', 'placeGroup_1', 'placeGroup_2', 'locno_1', 'locno_2',
       'deptime_1', 'arrtime_2', 'travtime_2', 'mode_2', 'locno_1',
       'loctype_1', 'state_1', 'country_1', 'state_fips_1', 'county_fips_1',
       'tract_fips_1', 'out_region_1', 'home_1', 'latitude_1', 'longitude_1',
       'locno_2', 'loctype_2', 'state_2', 'country_2', 'state_fips_2',
       'county_fips_2', 'tract_fips_2', 'out_region_2', 'home_2', 'latitude_2',
       'longitude_2', 'wtperfin', 'tract_1', 'tract_2'],
      dtype='object')

## Simplify Mode Representation

In [23]:
tp['mode'] = tp['mode_2']//100
tp['mode'] = tp['mode'].map({1:1,2:2,3:3,4:3,5:4,6:3,7:3})
print("1: Active; 2: Auto; 3: Mobility Services; 4: Public Transit")
print(tp.groupby('mode').sum()['wtperfin']/tp['wtperfin'].sum()*100)

1: Active; 2: Auto; 3: Mobility Services; 4: Public Transit
mode
1    13.281503
2    75.000827
3     5.135885
4     6.581785
Name: wtperfin, dtype: float64


In [24]:
o_mode = tp.groupby(['state_1', 'state_fips_1', 'county_fips_1', 'tract_fips_1', 'mode']).sum()['wtperfin'].reset_index()
o_mode['key'] = 0
o_mode_full = pd.merge(o_mode[['state_1', 'state_fips_1', 'county_fips_1', 'tract_fips_1','key']].drop_duplicates(), 
                       o_mode[['mode','key']].drop_duplicates())

o_mode = o_mode.drop("key",1)
o_mode_full = o_mode_full.drop("key",1)

o_mode = pd.merge(o_mode_full, o_mode, on=['state_1', 'state_fips_1', 'county_fips_1', 'tract_fips_1','mode'], how='left').fillna(0)

trip_generation = tp.groupby(['state_1', 'state_fips_1', 'county_fips_1',
       'tract_fips_1']).sum()['wtperfin'].reset_index()

o_mode = pd.merge(o_mode, trip_generation, on=['state_1', 'state_fips_1', 'county_fips_1', 'tract_fips_1'],
                 suffixes=("_mode","_all"))

o_mode.to_csv(data_dir+"origin_trip_behavior.csv", index=False)

In [26]:
o_mode = tp.groupby(['state_2', 'state_fips_2', 'county_fips_2', 'tract_fips_2', 'mode']).sum()['wtperfin'].reset_index()
o_mode['key'] = 0
o_mode_full = pd.merge(o_mode[['state_2', 'state_fips_2', 'county_fips_2', 'tract_fips_2','key']].drop_duplicates(), 
                       o_mode[['mode','key']].drop_duplicates())

o_mode = o_mode.drop("key",1)
o_mode_full = o_mode_full.drop("key",1)

o_mode = pd.merge(o_mode_full, o_mode, on=['state_2', 'state_fips_2', 'county_fips_2', 'tract_fips_2','mode'], how='left').fillna(0)

trip_generation = tp.groupby(['state_2', 'state_fips_2', 'county_fips_2',
       'tract_fips_2']).sum()['wtperfin'].reset_index()

o_mode = pd.merge(o_mode, trip_generation, on=['state_2', 'state_fips_2', 'county_fips_2', 'tract_fips_2'],
                 suffixes=("_mode","_all"))

o_mode.to_csv(data_dir+"destination_trip_behavior.csv", index=False)

## OD pairs

In [25]:
# Keep trips in Illinois only
tp = tp[(tp['state_fips_1']==17)&(tp['state_fips_2']==17)]

In [26]:
od_weighted = tp.groupby(['tract_1','tract_2'], as_index=False).sum()[['tract_1','tract_2','wtperfin']]
od_count = tp.groupby(['tract_1','tract_2'], as_index=False).count()[['tract_1','tract_2','sampno']]
od = pd.merge(od_weighted, od_count, on=['tract_1','tract_2'])

print("# OD pairs: %d \t # records: %d \t # trips: %d" %(len(od), od.sum()['sampno'], od.sum()['wtperfin']))
print("# OD pairs with more than 5 records: %d accounting for %d records and %d trips" %\
      (len(od[od['sampno']>5]), od[od['sampno']>5].sum()['sampno'], od[od['sampno']>5].sum()['wtperfin']))

# OD pairs: 46151 	 # records: 96698 	 # trips: 25859022
# OD pairs with more than 5 records: 2413 accounting for 32471 records and 7759413 trips


## OD Mode

In [27]:
od_mode = tp.groupby(['tract_1','tract_2','mode'], as_index=False).sum()[['tract_1','tract_2','mode','wtperfin']]

od_mode['key'] = 0
od['key'] = 0

# Create a df with all combinations of od and mode
od_mode_full = pd.merge(od[['tract_1','tract_2','key']].drop_duplicates(), od_mode[['mode','key']].drop_duplicates()).drop("key", 1)
od_mode = od_mode.drop("key",1)
od = od.drop("key",1)
od_mode_full["i"] = 1

In [28]:
od_mode = pd.merge(od_mode_full, od_mode, on=['tract_1','tract_2','mode'], how='outer').fillna(0)
od_mode = pd.merge(od_mode, od, on=['tract_1','tract_2'], suffixes=("_mode","_od"))
od_mode['mode_share'] = od_mode['wtperfin_mode'] / od_mode['wtperfin_od']

In [29]:
od_mode.to_csv(data_dir+"od_mode.csv", index=False)

## Filter OD pairs
- A lot of public transit trips are filltered.

In [30]:
od = od[od['sampno']>5]

In [31]:
tracts = pd.concat([od['tract_1'], od['tract_2']]).drop_duplicates().tolist()
print("# Census tracts in these OD pairs:", len(tracts))

# Census tracts in these OD pairs: 925


In [32]:
od_mode = tp.groupby(['tract_1','tract_2','mode'], as_index=False).count()[['tract_1','tract_2','mode','wtperfin']]

od_mode['key'] = 0
od['key'] = 0
# Use filtered OD (>5 trips)
od_mode_full = pd.merge(od[['tract_1','tract_2','key']].drop_duplicates(), od_mode[['mode','key']].drop_duplicates()).drop("key", 1)
od_mode = od_mode.drop("key",1)
od = od.drop("key",1)
od_mode_full["i"] = 1

In [33]:
od_mode = pd.merge(od_mode_full, od_mode, on=['tract_1','tract_2','mode'], how='outer').fillna(0)
od_mode = pd.merge(od_mode, od, on=['tract_1','tract_2'], suffixes=("_mode","_od"))
od_mode['mode_share'] = od_mode['wtperfin_mode'] / od_mode['wtperfin_od']

In [34]:
od_mode.to_csv(data_dir+"od_mode_filtered.csv", index=False)

In [36]:
# Output Census Tracts to Get Images
tracts = pd.concat([od['tract_1'], od['tract_2']]).drop_duplicates().tolist()
lc = pd.read_csv(survey_dir+"location.csv")
lc['tract'] = lc['state_fips']*1000000000+lc['county_fips']*1000000+lc['tract_fips']

lc = lc[['tract','state_fips','county_fips','tract_fips','latitude','longitude']].drop_duplicates()
lc = lc[lc['tract'].isin(tracts)]

lc['train_test'] = np.random.rand(len(lc)) < 0.1
lc['train_test'] = lc['train_test'].astype(int)
lc.to_csv(data_dir+"census_tracts_filtered.csv", index=False)